In [ ]:
import numpy as np
import pandas as pd
import torch

# Basic tensor operations

## 1D Tensors

### Construct LongTensor

In [ ]:
a = torch.tensor([1, 2, 3, 4])
print(a.type())  # type of Tensor
print(a.dtype)  # type of data within Tensor

### Methods to construct float Tensor

In [ ]:
a = torch.tensor([1, 2, 3], dtype=float)  # float64
a = torch.FloatTensor([1, 2, 3])  # float32
a = a.type(torch.FloatTensor)  # transform tensor into float32

### Informative methods

In [ ]:
a.size()
a.shape  # IDEM
a.ndimension()  # rank

### Transformative methods

#### Transform shape

In [ ]:
a.shape # initial shape (3)

# Transform in shape (3, 1)
a.view(3,1).shape
a.view(-1, 1).shape  # Idem

# Transform in shape (1, 3)
a.view(1, 3).shape
a.view(1, -1).shape

#### Transform type

In [ ]:
# Numpy & Torch
# -------------
numpy_arr = np.array([1, 2, 3])

# Numpy => Torch
tensor = torch.from_numpy(numpy_arr)
# Torch => Numpy
numpy_arr = tensor.numpy()

# ps: these are pointers, any change is 
# repercuted on the other variables

In [ ]:
# Pandas & Torch
# --------------
pd_series = pd.Series([1, 2, 3])

# Pandas => Torch
tensor = torch.from_numpy(pd_series.values)

# Torch => Pandas
pd_series = pd.Series(tensor)

In [ ]:
# Lists & Tensor
# --------------

# List => Tensor
tensor = torch.tensor(dumb_list)

# Tensor => List
dumb_list = tensor.tolist()